In [16]:
import numpy as np
import matplotlib.pyplot as plt
from sympy import *

In [15]:
mystery_func = lambda x: (x-40)**2 + 3*x + 72
mystery_func_prime = lambda x: 2 * x + 3

We want to minimize our mystery function, but we can't graph it. We can evaluate the function at different points, and we can also evaluate it's derivative.

Let's try some random values

In [14]:
mystery_func(5)

112

In [17]:
x = Symbol('x')

In [22]:
y = 2*x**2 + 1

In [23]:
yprime = y.diff(x)

In [30]:
yprime

4*x

In [29]:
lambdify(x, yprime)(5)

20